In [ ]:
pip install fastapi uvicorn transformers scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 5.9 MB/s eta 0:00:00


In [11]:
pip install fastapi uvicorn transformers torch scikit-learn pandas numpy

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [6]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np

app = FastAPI()

# sample data for RAG
articles = [
    {"title": "Dealing with Anxiety", "content": "Anxiety is a feeling of worry, nervousness, or unease about something..."},
    {"title": "Coping with Depression", "content": "Depression is more than just feeling sad. It's a serious mental health condition..."},

]

class RAGRequest(BaseModel):
    prompt: str

class ClassificationRequest(BaseModel):
    text: str

# Load an open-source LLM for text generation (e.g., GPT-2)
generator = pipeline('text-generation', model='gpt2')

# Dummy dataset for classification
data = pd.DataFrame({
    "text": ["I feel happy today", "I am very sad", "I am anxious about my exams", "I feel so depressed"],
    "category": ["happy", "sad", "anxious", "depressed"]
})

# TF-IDF Vectorizer for text data
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data["text"])

# Nearest Neighbors for RAG
nn_model = NearestNeighbors(n_neighbors=1, metric='cosine').fit(X)

# Load a pre-trained classification model (e.g., BERT for sequence classification)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)

# Mapping from label indices to categories
label_map = {0: "happy", 1: "sad", 2: "anxious", 3: "depressed"}

@app.post("/rag")
async def rag_endpoint(request: RAGRequest):
    prompt = request.prompt

    # Generate response using the LLM
    generated = generator(prompt, max_length=50, num_return_sequences=1)
    generated_text = generated[0]['generated_text']

    # Find the most relevant article using Nearest Neighbors
    query_vec = vectorizer.transform([prompt])
    distances, indices = nn_model.kneighbors(query_vec, n_neighbors=1)
    article = articles[indices[0][0]]

    return {"generated_text": generated_text, "relevant_article": article}

@app.post("/classification")
async def classification_endpoint(request: ClassificationRequest):
    text = request.text

    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    category = label_map[predicted_class]

    return {"text": text, "category": category}

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
pip install nest_asyncio

In [12]:
pip install uvicorn

In [2]:
pip install fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.5 MB/s eta 0:00:00


In [ ]:
import nest_asyncio
nest_asyncio.apply()

import uvicorn
uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [306]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [306]


In [ ]:
import threading
import uvicorn

def run_app():
    uvicorn.run(app, host="0.0.0.0", port=2870)

if __name__ == "__main__":
    thread = threading.Thread(target=run_app)
    thread.start()

Exception in thread Thread-13 (run_app):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner


In [ ]:
pip install fastapi uvicorn transformers sentence-transformers faiss-cpu

In [ ]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 36.3 MB/s eta 0:00:00


In [29]:
pip install fastapi[all] transformers sentence-transformers faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.0 MB/s eta 0:00:00


In [ ]:
import uvicorn

def start_app():
    uvicorn.run("app:app", host="0.0.0.0", port=63561, reload=True)

if __name__ == "__main__":
    start_app()

INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://0.0.0.0:63561 (Press CTRL+C to quit)
INFO:     Started reloader process [281] using WatchFiles


In [ ]:
FROM tiangolo/uvicorn-gunicorn-fastapi:python3.9

# Install your dependencies
COPY requirements.txt /app/
RUN pip install --no-cache-dir -r /app/requirements.txt

# Copy your application code
COPY . /app/

# Expose the port FastAPI will run on
EXPOSE 63561

# Start the FastAPI application
CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "63561"]

In [26]:
import asyncio

# Define your coroutine
async def my_coroutine():
    await asyncio.sleep(1)
    print("Coroutine finished")

# Run the coroutine
await my_coroutine()

Coroutine finished


In [23]:
pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 22.8 MB/s eta 0:00:00


In [28]:
import pytest
import asyncio

@pytest.mark.asyncio
async def test_my_coroutine():
    await asyncio.sleep(1)
    assert True

In [21]:
pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.2 MB/s eta 0:00:00


In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import torch
import nest_asyncio

app = FastAPI()

# Dummy data for RAG
articles = [
    {"title": "Dealing with Anxiety", "content": "Anxiety is a feeling of worry, nervousness, or unease about something..."},
    {"title": "Coping with Depression", "content": "Depression is more than just feeling sad. It's a serious mental health condition..."},
    # Add more articles as needed
]

class RAGRequest(BaseModel):
    prompt: str

class ClassificationRequest(BaseModel):
    text: str

# Load an open-source LLM for text generation (e.g., GPT-2)
generator = pipeline('text-generation', model='gpt2')

# Load Sentence-Transformers model for embedding generation
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

# Create FAISS index
article_embeddings = sentence_model.encode([article['content'] for article in articles])
faiss_index = faiss.IndexFlatL2(article_embeddings.shape[1])
faiss_index.add(article_embeddings)

# Load a pre-trained classification model (e.g., BERT for sequence classification)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)

# Mapping from label indices to categories
label_map = {0: "happy", 1: "sad", 2: "anxious", 3: "depressed"}

@app.post("/rag")
async def rag_endpoint(request: RAGRequest):
    prompt = request.prompt

    # Generate response using the LLM
    generated = generator(prompt, max_length=50, num_return_sequences=1)
    generated_text = generated[0]['generated_text']

    # Find the most relevant article using FAISS
    query_embedding = sentence_model.encode([prompt])
    _, indices = faiss_index.search(query_embedding, k=1)
    article = articles[indices[0][0]]

    return {"generated_text": generated_text, "relevant_article": article}

@app.post("/classification")
async def classification_endpoint(request: ClassificationRequest):
    text = request.text

    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    category = label_map[predicted_class]

    return {"text": text, "category": category}

if __name__ == "__main__":
    import uvicorn
    nest_asyncio.apply()
    uvicorn.run(app, host="0.0.0.0", port=8000)

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-7' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 577, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/lib/python3.10/asyncio/tasks.py", l

In [ ]:
pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.1 MB/s eta 0:00:00


In [ ]:
pip install nest_asyncio

In [ ]:
pip install sentence-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu